In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from keras import models, layers
import seaborn as sns

filepath = 'international-airline-passengers.csv'

In [13]:
dataset = pd.read_csv(filepath, delimiter=";")

In [14]:
dataset.head()

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [22]:
dataset.shape

(144, 2)